<center><h1 style="font-size:3em"> Improving Few-shot Text Classification </h1></center>
<br>
<center><i style="font-size:1.3em">via Pretrained Language Representations</i></center>

Resources :
- Includes Easy Data Augmentation (EDA) package : https://github.com/jasonwei20/eda_nlp/blob/master/code/eda.py introduced after this paper : https://arxiv.org/pdf/1901.11196.pdf
- Neural Structure Learning for sentiment classification (learn graph from embedding) : https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb
- RNN text classification with Tensorflow 2.0 : https://www.tensorflow.org/beta/tutorials/text/text_classification_rnn
- Great paper from August 2019 : https://arxiv.org/pdf/1908.08788.pdf


In [1]:
import pandas as pd
import numpy as np
from random import seed
from random import sample
import random
from random import shuffle
import re

seed(42)
np.random.seed(42)

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import gensim.downloader as api
from gensim.models.keyedvectors import Word2VecKeyedVectors

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy import spatial

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet 

import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow.keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

import neural_structured_learning as nsl
import tensorflow_hub as hub

import torch
import torch.nn as nn
#import torch.legacy.nn as luann
import sys

from torchtext.data import Field
from torchtext.data import Dataset
import torch
import torch.utils.data
from torch.autograd import Variable
from torchtext.vocab import Vocab

from collections import Counter
from collections import OrderedDict

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import (
        BertModel,
        BertForNextSentencePrediction,
        BertForMaskedLM,
        BertForMultipleChoice,
        BertForPreTraining,
        BertForQuestionAnswering,
        BertForSequenceClassification,
        BertForTokenClassification,
        )

/anaconda3/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.21.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
'ours', 'ourselves', 'you', 'your', 'yours', 
'yourself', 'yourselves', 'he', 'him', 'his', 
'himself', 'she', 'her', 'hers', 'herself', 
'it', 'its', 'itself', 'they', 'them', 'their', 
'theirs', 'themselves', 'what', 'which', 'who', 
'whom', 'this', 'that', 'these', 'those', 'am', 
'is', 'are', 'was', 'were', 'be', 'been', 'being', 
'have', 'has', 'had', 'having', 'do', 'does', 'did',
'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
'because', 'as', 'until', 'while', 'of', 'at', 
'by', 'for', 'with', 'about', 'against', 'between',
'into', 'through', 'during', 'before', 'after', 
'above', 'below', 'to', 'from', 'up', 'down', 'in',
'out', 'on', 'off', 'over', 'under', 'again', 
'further', 'then', 'once', 'here', 'there', 'when', 
'where', 'why', 'how', 'all', 'any', 'both', 'each', 
'few', 'more', 'most', 'other', 'some', 'such', 'no', 
'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
'very', 's', 't', 'can', 'will', 'just', 'don', 
'should', 'now', '']

In [3]:
#model2 = api.load('glove-twitter-25')
model2 = api.load('word2vec-google-news-300')

# Load the data

The dataset comes from Stackoverflow Short Text Classification : https://github.com/jacoxu/StackOverflow

In [4]:
df = pd.read_csv("stack.csv")

In [5]:
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

In [6]:
df['Text'] = df['Text'].apply(lambda x: get_only_chars(x))

In [7]:
df.head()

,Text,Label
0,how do i fill a dataset or a datatable from a ...,18
1,how do you page a collection with linq,18
2,best subversion clients for windows vista bit,3
3,best practice collaborative environment bin di...,3
4,visual studio setup project per user registry ...,7


# Prepare the data

In [8]:
num_classes = 2
sample_size = 50

In [9]:
# Generate samples that contains K samples of each class

def gen_sample(sample_size, num_classes):
    
    df_1 = df[(df["Label"]<num_classes + 1)].reset_index().drop(["index"], axis=1).reset_index().drop(["index"], axis=1)
    train = df_1[df_1["Label"] == np.unique(df_1['Label'])[0]].sample(sample_size)
    
    train_index = train.index.tolist()
    
    for i in range(1,num_classes):
        train_2 = df_1[df_1["Label"] == np.unique(df_1['Label'])[i]].sample(sample_size)
        train = pd.concat([train, train_2], axis=0)
        train_index.extend(train_2.index.tolist())
        
    test = df_1[~df_1.index.isin(train_index)]

    return train, test

In [10]:
# Text processing (split, find token id, get embedidng)

def transform_sentence(text, model):
    
    """
    Mean embedding vector
    """
    
    def preprocess_text(raw_text, model=model):
        
        """ 
        Eexcluding unknown words and get corresponding token
        """
        
        raw_text = raw_text.split()
        
        return list(filter(lambda x: x in model.vocab, raw_text))
    
    tokens = preprocess_text(text)
    
    if not tokens:
        return np.zeros(model.vector_size)

    text_vector = np.mean(model[tokens], axis=0)
    
    return np.array(text_vector)

In [11]:
train, test = gen_sample(sample_size, num_classes)

In [12]:
train

,Text,Label
1521,how to get wordpress page id after looping posts,1
1737,using wp query to pull content from a specific...,1
1740,wordpress how to show just posts on main index,1
1660,wordpress is it possible to make one particula...,1
1411,exclude templates in wordpress page,1
1678,wierd date and time formating in wordpress y m d,1
1626,wordpress custom post type templates,1
1513,inserting wordpress plugin content to posts,1
1859,how can i delay one feed in wordpress but not ...,1
1072,how can i remove jquery from the frontside of ...,1


In [13]:
train.shape

(100, 2)

In [14]:
X_train = train['Text']
y_train = train['Label'].values
X_test = test['Text']
y_test = test['Label'].values

X_train_mean = X_train.apply(lambda x : transform_sentence(x, model2))
X_test_mean = X_test.apply(lambda x : transform_sentence(x, model2))

X_train_mean = pd.DataFrame(X_train_mean)['Text'].apply(pd.Series)
X_test_mean = pd.DataFrame(X_test_mean)['Text'].apply(pd.Series)

# Data Augmentation

## Replace words

In [90]:
def get_synonyms(word):
    
    synonyms = set()
    
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            #print("replaced", random_word, "with", synonym)
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

def iterative_replace(df):
    
    df = df.reset_index().drop(['index'], axis=1)
    index_row = df.index
    df_2 = pd.DataFrame()
    
    for row in index_row:
        for k in range(1,6):
            df_2 = df_2.append({'Text':synonym_replacement(df.loc[row]['Text'], k), 'Label':df.loc[row]['Label']}, ignore_index=True)
    return df_2

## Delete words

In [91]:
def random_deletion(words, p):

    words = words.split()
    
    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    sentence = ' '.join(new_words)
    
    return sentence

def iterative_delete(df):
    
    df = df.reset_index().drop(['index'], axis=1)
    index_row = df.index
    df_2 = pd.DataFrame()
    
    for row in index_row:
        df_2 = df_2.append({'Text':random_deletion(df.loc[row]['Text'], 0.25), 'Label':df.loc[row]['Label']}, ignore_index=True)
    return df_2

## Random Swap

In [92]:
def random_swap(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        new_words = swap_word(new_words)
        
    sentence = ' '.join(new_words)
    
    return sentence

def swap_word(new_words):
    
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        
        if counter > 3:
            return new_words
    
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

def iterative_swap(df):
    
    df = df.reset_index().drop(['index'], axis=1)
    index_row = df.index
    df_2 = pd.DataFrame()
    for row in index_row:
        df_2 = df_2.append({'Text':random_swap(df.loc[row]['Text'], 2), 'Label':df.loc[row]['Label']}, ignore_index=True)
    return df_2

## Random Insertion

In [93]:
def random_insertion(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        add_word(new_words)
        
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    
    synonyms = []
    counter = 0
    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)
    
def iterative_insert(df):
    
    df = df.reset_index().drop(['index'], axis=1)
    index_row = df.index
    df_2 = pd.DataFrame()
    
    for row in index_row:
        df_2 = df_2.append({'Text':random_insertion(df.loc[row]['Text'], 2), 'Label':df.loc[row]['Label']}, ignore_index=True)
        
    return df_2

## Data Augmentation

In [94]:
df_replace = iterative_replace(train)
df_delete = iterative_delete(train)
df_swap = iterative_swap(train)
df_insert = iterative_insert(train)

train = pd.concat([train, df_replace, df_delete, df_swap, df_insert], axis=0).reset_index().drop(['index'], axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [95]:
X_train = train['Text']
y_train = train['Label'].values
X_test = test['Text']
y_test = test['Label'].values

X_train_mean = X_train.apply(lambda x : transform_sentence(x, model2))
X_test_mean = X_test.apply(lambda x : transform_sentence(x, model2))

X_train_mean = pd.DataFrame(X_train_mean)['Text'].apply(pd.Series)
X_test_mean = pd.DataFrame(X_test_mean)['Text'].apply(pd.Series)

# Utilities in PyTorch 

## CNN Model

In [16]:
class MaxPool(nn.Module):
    def __init__(self, dim=1):
        super(MaxPool, self).__init__()
        self.dim = dim
    
    def forward(self, input):
        return torch.max(input, self.dim)[0]

    def __repr__(self):
        return self.__class__.__name__ +'('+ 'dim=' + str(self.dim) + ')'

class View(nn.Module):
    def __init__(self, *sizes):
        super(View, self).__init__()
        self.sizes_list = sizes

    def forward(self, input):
        return input.view(*self.sizes_list)

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'sizes=' + str(self.sizes_list) + ')'

class Transpose(nn.Module):
    def __init__(self, dim1=0, dim2=1):
        super(Transpose, self).__init__()
        self.dim1 = dim1
        self.dim2 = dim2

    def forward(self, input):
        return input.transpose(self.dim1, self.dim2).contiguous()

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'between=' + str(self.dim1) + ',' + str(self.dim2) + ')'

class CNNModel(nn.Module):
    def __init__(self, vocab_size, num_labels, emb_size, w_hid_size, h_hid_size, win, batch_size,with_proj=False):
        super(CNNModel, self).__init__()

        self.model = nn.Sequential()
        self.model.add_module('transpose', Transpose())
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_size)
        self.model.add_module('emb', self.embed)
        if with_proj:
            self.model.add_module('view1', View(-1, emb_size))
            self.model.add_module('linear1', nn.Linear(emb_size, w_hid_size))
            self.model.add_module('relu1', nn.ReLU())
        else:
            w_hid_size = emb_size

        self.model.add_module('trans2', Transpose(1, 2))

        conv_nn = nn.Conv1d(w_hid_size, h_hid_size, win, padding=1)
        self.model.add_module('conv', conv_nn)
        self.model.add_module('relu2', nn.ReLU())

        self.model.add_module('max', MaxPool(2))

        self.model.add_module('view4', View(-1, h_hid_size))
        self.model.add_module('linear2', nn.Linear(h_hid_size, num_labels))
        self.model.add_module('softmax', nn.LogSoftmax())


    def forward(self, x):

        output = self.model.forward(x)

        return output

## Load BERT

In [17]:
dependencies = ['torch', 'tqdm', 'boto3', 'requests', 'regex']

def bertTokenizer(*args, **kwargs):
    tokenizer = BertTokenizer.from_pretrained(*args, **kwargs)
    return tokenizer

def bertModel(*args, **kwargs):
    model = BertModel.from_pretrained(*args, **kwargs)
    return model

def bertForNextSentencePrediction(*args, **kwargs):
    model = BertForNextSentencePrediction.from_pretrained(*args, **kwargs)
    return model

def bertForPreTraining(*args, **kwargs):
    model = BertForPreTraining.from_pretrained(*args, **kwargs)
    return model

def bertForMaskedLM(*args, **kwargs):
    model = BertForMaskedLM.from_pretrained(*args, **kwargs)
    return model

def bertForSequenceClassification(*args, **kwargs):
    model = BertForSequenceClassification.from_pretrained(*args, **kwargs)
    return model

def bertForMultipleChoice(*args, **kwargs):
    model = BertForMultipleChoice.from_pretrained(*args, **kwargs)
    return model

def bertForQuestionAnswering(*args, **kwargs):
    model = BertForQuestionAnswering.from_pretrained(*args, **kwargs)
    return model

def bertForTokenClassification(*args, **kwargs):
    model = BertForTokenClassification.from_pretrained(*args, **kwargs)
    return model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


## Bert Fine-Tuning

In [ ]:
import argparse
import logging
import os
import random
from io import open

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from pytorch_pretrained_bert.modeling import BertForPreTraining
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from torch.nn import CrossEntropyLoss


class BERTDataset(Dataset):
    def __init__(self, corpus_path, tokenizer, seq_len, encoding="utf-8", corpus_lines=None, on_memory=True):
        self.vocab = tokenizer.vocab
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.on_memory = on_memory
        self.corpus_lines = corpus_lines  # number of non-empty lines in input corpus
        self.corpus_path = corpus_path
        self.encoding = encoding
        self.current_doc = 0  # to avoid random sentence from same doc

        # for loading samples directly from file
        self.sample_counter = 0  # used to keep track of full epochs on file
        self.line_buffer = None  # keep second sentence of a pair in memory and use as first sentence in next pair

        # for loading samples in memory
        self.current_random_doc = 0
        self.num_docs = 0
        self.sample_to_doc = [] # map sample index to doc and line

        # load samples into memory
        if on_memory:
            self.all_docs = []
            doc = []
            self.corpus_lines = 0
            with open(corpus_path, "r", encoding=encoding) as f:
                for line in tqdm(f, desc="Loading Dataset", total=corpus_lines):
                    line = line.strip()
                    if line == "":
                        self.all_docs.append(doc)
                        doc = []
                        #remove last added sample because there won't be a subsequent line anymore in the doc
                        self.sample_to_doc.pop()
                    else:
                        #store as one sample
                        sample = {"doc_id": len(self.all_docs),
                                  "line": len(doc)}
                        self.sample_to_doc.append(sample)
                        doc.append(line)
                        self.corpus_lines = self.corpus_lines + 1

            # if last row in file is not empty
            if self.all_docs[-1] != doc:
                self.all_docs.append(doc)
                self.sample_to_doc.pop()

            self.num_docs = len(self.all_docs)

        # load samples later lazily from disk
        else:
            if self.corpus_lines is None:
                with open(corpus_path, "r", encoding=encoding) as f:
                    self.corpus_lines = 0
                    for line in tqdm(f, desc="Loading Dataset", total=corpus_lines):
                        if line.strip() == "":
                            self.num_docs += 1
                        else:
                            self.corpus_lines += 1

                    # if doc does not end with empty line
                    if line.strip() != "":
                        self.num_docs += 1

            self.file = open(corpus_path, "r", encoding=encoding)
            self.random_file = open(corpus_path, "r", encoding=encoding)

    def __len__(self):
        # last line of doc won't be used, because there's no "nextSentence". Additionally, we start counting at 0.
        return self.corpus_lines - self.num_docs - 1

    def __getitem__(self, item):
        cur_id = self.sample_counter
        self.sample_counter += 1
        if not self.on_memory:
            # after one epoch we start again from beginning of file
            if cur_id != 0 and (cur_id % len(self) == 0):
                self.file.close()
                self.file = open(self.corpus_path, "r", encoding=self.encoding)

        t1, t2, is_next_label = self.random_sent(item)

        # tokenize
        tokens_a = self.tokenizer.tokenize(t1)
        tokens_b = self.tokenizer.tokenize(t2)

        # combine to one sample
        cur_example = InputExample(guid=cur_id, tokens_a=tokens_a, tokens_b=tokens_b, is_next=is_next_label)

        # transform sample to features
        cur_features = convert_example_to_features(cur_example, self.seq_len, self.tokenizer)

        cur_tensors = (torch.tensor(cur_features.input_ids),
                       torch.tensor(cur_features.input_mask),
                       torch.tensor(cur_features.segment_ids),
                       torch.tensor(cur_features.lm_label_ids),
                       torch.tensor(cur_features.is_next))

        return cur_tensors

    def random_sent(self, index):
        """
        Get one sample from corpus consisting of two sentences. With prob. 50% these are two subsequent sentences
        from one doc. With 50% the second sentence will be a random one from another doc.
        :param index: int, index of sample.
        :return: (str, str, int), sentence 1, sentence 2, isNextSentence Label
        """
        t1, t2 = self.get_corpus_line(index)
        t = random.random()
        if t > 0.5:
            label = 0
        else:
            t2 = self.get_random_line()
            label = 1

        assert len(t1) > 0
        assert len(t2) > 0
        return t1, t2, label

    def get_corpus_line(self, item):
        """
        Get one sample from corpus consisting of a pair of two subsequent lines from the same doc.
        :param item: int, index of sample.
        :return: (str, str), two subsequent sentences from corpus
        """
        t1 = ""
        t2 = ""
        assert item < self.corpus_lines
        if self.on_memory:
            sample = self.sample_to_doc[item]
            t1 = self.all_docs[sample["doc_id"]][sample["line"]]
            t2 = self.all_docs[sample["doc_id"]][sample["line"]+1]
            # used later to avoid random nextSentence from same doc
            self.current_doc = sample["doc_id"]
            return t1, t2
        else:
            if self.line_buffer is None:
                # read first non-empty line of file
                while t1 == "" :
                    t1 = next(self.file).strip()
                    t2 = next(self.file).strip()
            else:
                # use t2 from previous iteration as new t1
                t1 = self.line_buffer
                t2 = next(self.file).strip()
                # skip empty rows that are used for separating documents and keep track of current doc id
                while t2 == "" or t1 == "":
                    t1 = next(self.file).strip()
                    t2 = next(self.file).strip()
                    self.current_doc = self.current_doc+1
            self.line_buffer = t2

        assert t1 != ""
        assert t2 != ""
        return t1, t2

    def get_random_line(self):
        """
        Get random line from another document for nextSentence task.
        :return: str, content of one line
        """
        # Similar to original tf repo: This outer loop should rarely go for more than one iteration for large
        # corpora. However, just to be careful, we try to make sure that
        # the random document is not the same as the document we're processing.
        for _ in range(10):
            if self.on_memory:
                rand_doc_idx = random.randint(0, len(self.all_docs)-1)
                rand_doc = self.all_docs[rand_doc_idx]
                line = rand_doc[random.randrange(len(rand_doc))]
            else:
                rand_index = random.randint(1, self.corpus_lines if self.corpus_lines < 1000 else 1000)
                # rand_index = 892
                #pick random line
                for _ in range(rand_index):
                    line = self.get_next_line()
                
            #check if our picked random line is really from another doc like we want it to be
            if self.current_random_doc != self.current_doc:
                
                break
        # print("random Index:", rand_index, line)
        return line

    def get_next_line(self):
        """ Gets next line of random_file and starts over when reaching end of file"""
        try:
            line = next(self.random_file).strip()

            #keep track of which document we are currently looking at to later avoid having the same doc as t1
            while line == "":
                self.current_random_doc = self.current_random_doc + 1
                line = next(self.random_file).strip()
        except StopIteration:
            self.random_file.close()
            self.random_file = open(self.corpus_path, "r", encoding=self.encoding)
            line = next(self.random_file).strip()
        
        return line


class InputExample(object):
    """A single training/test example for the language model."""

    def __init__(self, guid, tokens_a, tokens_b=None, is_next=None, lm_labels=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            tokens_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            tokens_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.tokens_a = tokens_a
        self.tokens_b = tokens_b
        self.is_next = is_next  # nextSentence
        self.lm_labels = lm_labels  # masked words for language model


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, is_next, lm_label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.is_next = is_next
        self.lm_label_ids = lm_label_ids


def random_word(tokens, tokenizer):
    """
    Masking some random tokens for Language Model task with probabilities as in the original BERT paper.
    :param tokens: list of str, tokenized sentence.
    :param tokenizer: Tokenizer, object used for tokenization (we need it's vocab here)
    :return: (list of str, list of int), masked tokens and related labels for LM prediction
    """
    output_label = []

    for i, token in enumerate(tokens):
        prob = random.random()
        # mask token with 15% probability
        if prob < 0.15:
            prob /= 0.15

            # 80% randomly change token to mask token
            if prob < 0.8:
                tokens[i] = "[MASK]"

            # 10% randomly change token to random token
            elif prob < 0.9:
                tokens[i] = random.choice(list(tokenizer.vocab.items()))[0]

            # -> rest 10% randomly keep current token

            # append current token to output (we will predict these later)
            try:
                output_label.append(tokenizer.vocab[token])
            except KeyError:
                # For unknown words (should not occur with BPE vocab)
                output_label.append(tokenizer.vocab["[UNK]"])
                logger.warning("Cannot find token '{}' in vocab. Using [UNK] insetad".format(token))
        else:
            # no masking token (will be ignored by loss function later)
            output_label.append(-1)

    return tokens, output_label


def convert_example_to_features(example, max_seq_length, tokenizer):
    """
    Convert a raw sample (pair of sentences as tokenized strings) into a proper training sample with
    IDs, LM labels, input_mask, CLS and SEP tokens etc.
    :param example: InputExample, containing sentence input as strings and is_next label
    :param max_seq_length: int, maximum length of sequence.
    :param tokenizer: Tokenizer
    :return: InputFeatures, containing all inputs and labels of one sample as IDs (as used for model training)
    """
    tokens_a = example.tokens_a
    tokens_b = example.tokens_b
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

    tokens_a, t1_label = random_word(tokens_a, tokenizer)
    tokens_b, t2_label = random_word(tokens_b, tokenizer)
    # concatenate lm labels and account for CLS, SEP, SEP
    lm_label_ids = ([-1] + t1_label + [-1] + t2_label + [-1])

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambigiously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    assert len(tokens_b) > 0
    for token in tokens_b:
        tokens.append(token)
        segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        lm_label_ids.append(-1)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    assert len(lm_label_ids) == max_seq_length

    if example.guid < 5:
        logger.info("*** Example ***")
        logger.info("guid: %s" % (example.guid))
        logger.info("tokens: %s" % " ".join(
                [str(x) for x in tokens]))
        logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        logger.info(
                "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        logger.info("LM label: %s " % (lm_label_ids))
        logger.info("Is next sentence label: %s " % (example.is_next))

    features = InputFeatures(input_ids=input_ids,
                             input_mask=input_mask,
                             segment_ids=segment_ids,
                             lm_label_ids=lm_label_ids,
                             is_next=example.is_next)
    return features

## Build Vocabulary

In [ ]:
class MTLField(Field):

    def __init__(
            self, **kwargs):
        super(MTLField, self).__init__(**kwargs)

    def build_vocab(self, dataset_list, **kwargs):
        ## Load BERT
        counter = Counter()
        sources = []
        for arg in dataset_list:
            if isinstance(arg, Dataset):
                sources += [getattr(arg, name) for name, field in
                            arg.fields.items() if field is self]
            else:
                sources.append(arg)
        for data in sources:
            for x in data:
                if not self.sequential:
                    x = [x]
                counter.update(x)
        specials = list(OrderedDict.fromkeys(
            tok for tok in [self.pad_token, self.init_token, self.eos_token]
            if tok is not None))
        self.vocab = Vocab(counter, specials=specials, **kwargs)

## MAML CNN Classifier

In [ ]:
import sys, os, glob, random
import time
import parser
import torch
import torch.nn as nn
# from AdaAdam import AdaAdam
import torch.optim as OPT
import numpy as np
from copy import deepcopy
from tqdm import tqdm, trange
import logging

from torchtext import data
import DataProcessing
from DataProcessing.MLTField import MTLField
from DataProcessing.NlcDatasetSingleFile import NlcDatasetSingleFile
from CNNModel import CNNModel


logger = logging.getLogger(__name__)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO )
batch_size = 10
seed = 12345678
torch.manual_seed(seed)
Train = False


device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
n_gpu = torch.cuda.device_count()
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

def load_train_test_files(listfilename, test_suffix='.test'):
    filein = open(listfilename, 'r')
    file_tuples = []
    task_classes = ['.t2', '.t4', '.t5']
    for line in filein:
        array = line.strip().split('\t')
        line = array[0]
        for t_class in task_classes:
            trainfile = line + t_class + '.train'
            devfile = line + t_class + '.dev'
            testfile = line + t_class + test_suffix
            file_tuples.append((trainfile, devfile, testfile))
    filein.close()
    return file_tuples

filelist = 'data/Amazon_few_shot/workspace.filtered.list'
targetlist = 'data/Amazon_few_shot/workspace.target.list'
workingdir = 'data/Amazon_few_shot'
emfilename = 'glove.6B.300d'
emfiledir = '..'

datasets = []
list_datasets = []


file_tuples = load_train_test_files(filelist)
print(file_tuples)

TEXT = MTLField(lower=True)
for (trainfile, devfile, testfile) in file_tuples:
    print(trainfile, devfile, testfile)
    LABEL1 = data.Field(sequential=False)
    train1, dev1, test1 = NlcDatasetSingleFile.splits(
        TEXT, LABEL1, path=workingdir, train=trainfile,
        validation=devfile, test=testfile)
    datasets.append((TEXT, LABEL1, train1, dev1, test1))
    list_datasets.append(train1)
    list_datasets.append(dev1)
    list_datasets.append(test1)

target_datasets = []
target_file = load_train_test_files(targetlist)
print(target_file)

for (trainfile, devfile, testfile) in target_file:
    print(trainfile, devfile, testfile)
    LABEL2 = data.Field(sequential=False)
    train2, dev2, test2 = NlcDatasetSingleFile.splits(TEXT, LABEL2, path=workingdir, 
    train=trainfile,validation=devfile, test=testfile)
    target_datasets.append((TEXT, LABEL2, train2, dev2, test2))

    

datasets_iters = []
for (TEXT, LABEL, train, dev, test) in datasets:
    train_iter, dev_iter, test_iter = data.BucketIterator.splits(
        (train, dev, test), batch_size=batch_size, device=device,shuffle=True)
    train_iter.repeat = False
    datasets_iters.append((train_iter, dev_iter, test_iter))

fsl_ds_iters = []
for (TEXT, LABEL, train, dev, test) in target_datasets:
    train_iter, dev_iter, test_iter = data.BucketIterator.splits(
        (train,dev, test), batch_size=batch_size, device=device)
    train_iter.repeat = False
    fsl_ds_iters.append((train_iter, dev_iter, test_iter))

num_batch_total = 0
for i, (TEXT, LABEL, train, dev, test) in enumerate(datasets):
    # print('DATASET%d'%(i+1))
    # print('train.fields', train.fields)
    # print('len(train)', len(train))
    # print('len(dev)', len(dev))
    # print('len(test)', len(test))
    # print('vars(train[0])', vars(train[0]))
    num_batch_total += len(train) / batch_size

TEXT.build_vocab(list_datasets, vectors = emfilename, vectors_cache = emfiledir)
# TEXT.build_vocab(list_dataset)

# build the vocabulary
for taskid, (TEXT, LABEL, train, dev, test) in enumerate(datasets):
    LABEL.build_vocab(train, dev, test)
    LABEL.vocab.itos = LABEL.vocab.itos[1:]

    for k, v in LABEL.vocab.stoi.items():
        LABEL.vocab.stoi[k] = v - 1

    # print vocab information
    # print('len(TEXT.vocab)', len(TEXT.vocab))
    # print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

    # print(LABEL.vocab.itos)
    # print(len(LABEL.vocab.itos))

    # print(len(LABEL.vocab.stoi))
fsl_num_tasks = 0
for taskid, (TEXT, LABEL, train, dev, test) in enumerate(target_datasets):
    fsl_num_tasks += 1
    LABEL.build_vocab(train, dev, test)
    LABEL.vocab.itos = LABEL.vocab.itos[1:]
    for k, v in LABEL.vocab.stoi.items():
        LABEL.vocab.stoi[k] = v - 1

nums_embed = len(TEXT.vocab)
dim_embed = 100
dim_w_hid = 200
dim_h_hid = 100
Inner_lr = 2e-6
Outer_lr = 1e-5

n_labels = []
for (TEXT, LABEL, train, dev, test) in datasets:
   n_labels.append(len(LABEL.vocab))
print(n_labels)
num_tasks = len(n_labels)
print("num_tasks", num_tasks)
winsize = 3
num_labels = len(LABEL.vocab.itos)
model = CNNModel(nums_embed, num_labels, dim_embed, dim_w_hid, dim_h_hid, winsize, batch_size)

print("GPU Device: ", device)
model.to(device)
print(model)

criterion = nn.CrossEntropyLoss()
opt = OPT.Adam(model.parameters(), lr=Inner_lr)
Inner_epochs = 4
epochs = 2

N_task = 5

task_list = np.arange(num_tasks)
print("Total Batch: ", num_batch_total)
output_model_file = '/tmp/CNN_MAML_output'
if Train:
    for t in trange(int(num_batch_total*epochs/Inner_epochs), desc="Iterations"):
        selected_task = np.random.choice(task_list, N_task,replace=False)
        weight_before = deepcopy(model.state_dict())
        update_vars = []
        fomaml_vars = []
        for task_id in selected_task:
            # print(task_id)
            (train_iter, dev_iter, test_iter) = datasets_iters[task_id]
            train_iter.init_epoch()
            model.train()
            n_correct = 0
            n_step = 0
            for inner_iter in range(Inner_epochs):
                batch = next(iter(train_iter))

                # print(batch.text)
                # print(batch.label)
                logits = model(batch.text)
                loss = criterion(logits.view(-1, num_labels), batch.label.data.view(-1))
                

                n_correct = (torch.max(logits, 1)[1].view(batch.label.size()).data == batch.label.data).sum()
                n_step = batch.batch_size
                loss.backward()
                opt.step()
                opt.zero_grad()
            task_acc = 100.*n_correct/n_step
            if t%10 == 0:
                logger.info("Iter: %d, task id: %d, train acc: %f", t, task_id, task_acc)
            weight_after = deepcopy(model.state_dict())
            update_vars.append(weight_after)
            model.load_state_dict(weight_before)

        new_weight_dict = {}
        for name in weight_before:
            weight_list = [tmp_weight_dict[name] for tmp_weight_dict in update_vars]
            weight_shape = list(weight_list[0].size())
            stack_shape = [len(weight_list)] + weight_shape
            stack_weight = torch.empty(stack_shape)
            for i in range(len(weight_list)):
                stack_weight[i,:] = weight_list[i] 
            new_weight_dict[name] = torch.mean(stack_weight, dim=0).cuda()
            new_weight_dict[name] = weight_before[name]+(new_weight_dict[name]-weight_before[name])/Inner_lr*Outer_lr
        model.load_state_dict(new_weight_dict)


    torch.save(model.state_dict(), output_model_file)

model.load_state_dict(torch.load(output_model_file))
logger.info("***** Running evaluation *****")
fsl_task_list = np.arange(fsl_num_tasks)
weight_before = deepcopy(model.state_dict())
fsl_epochs = 3
Total_acc = 0
opt = OPT.Adam(model.parameters(), lr=3e-4)

for task_id in fsl_task_list:
    model.train()
    (train_iter, dev_iter, test_iter) = fsl_ds_iters[task_id]
    train_iter.init_epoch()
    batch = next(iter(train_iter))
    for i in range(fsl_epochs):
        logits = model(batch.text)
        loss = criterion(logits.view(-1, num_labels), batch.label.data.view(-1))
        n_correct = (torch.max(logits, 1)[1].view(batch.label.size()).data == batch.label.data).sum()
        n_size = batch.batch_size
        train_acc = 100. * n_correct / n_size
        loss = criterion(logits.view(-1, num_labels), batch.label.data.view(-1))
        loss.backward()
        opt.step()
        opt.zero_grad()
        logger.info("  Task id: %d, fsl epoch: %d, Acc: %f, loss: %f", task_id, i, train_acc, loss)

    model.eval()
    test_iter.init_epoch()
    n_correct = 0
    n_size = 0
    for test_batch_idx, test_batch in enumerate(test_iter):
        with torch.no_grad():
            logits = model(test_batch.text)
        loss = criterion(logits.view(-1, num_labels), test_batch.label.data.view(-1))
        n_correct += (torch.max(logits, 1)[1].view(test_batch.label.size()).data == test_batch.label.data).sum()
        n_size += test_batch.batch_size
    test_acc = 100.* n_correct/n_size
    logger.info("FSL test Number: %d, Accuracy: %f",n_size, test_acc)
    Total_acc += test_acc
    model.load_state_dict(weight_before)

print("Mean Accuracy is : ", float(Total_acc)/fsl_num_tasks)
